# Validación de Datos: Tablas EUROSTAT

Este notebook valida la calidad, integridad y coherencia de **TODAS las tablas EUROSTAT** generadas por el proceso ETL.

## Arquitectura Modular

- **Este módulo**: Validación de tablas EUROSTAT (Oficina Europea de Estadística)
- **Módulo hermano**: `02a_validacion_INE.ipynb` (tablas INE)
- **Módulo de integración**: `02c_validacion_integracion.ipynb` (coherencia entre fuentes)
- **Orquestador**: `02_run_validation.py` (ejecuta todos los módulos)

## Framework de Validación

Usa el framework reutilizable en `utils/validation_framework.py` con reglas declarativas en `utils/validation_rules.py`.

## Salida

- **Reportes JSON/CSV**: `data/validated/logs/`
- **NO crea tablas** en SQL Server (las tablas originales permanecen intactas)

---

In [1]:
# 0. CONFIGURACIÓN Y FRAMEWORK
import sys
import pandas as pd
import pyodbc
from datetime import datetime

# Añadir utils al path
sys.path.append("../../")

# Importar framework de validación
from utils.validation_framework import (
    ValidationReport,
    check_schema,
    check_uniqueness,
    check_nulls,
    check_conditional_nulls,
    check_range,
    check_year_continuity,
)
from utils.validation_rules import get_rules, EUROSTAT_VALIDATION_RULES
from utils.config import DB_CONNECTION_STRING, MAX_NULL_PERCENT

# Conexión a SQL Server
conn = pyodbc.connect(DB_CONNECTION_STRING)

print("=" * 80)
print("MÓDULO DE VALIDACIÓN: TABLAS EUROSTAT")
print("=" * 80)
print("✅ Framework de validación cargado")
print(f"📊 Tablas EUROSTAT configuradas: {len(EUROSTAT_VALIDATION_RULES)}")
print(f"🔧 Umbrales: MAX_NULL_PERCENT={MAX_NULL_PERCENT*100}%")
print(f"⏰ Timestamp: {datetime.now().isoformat()}")

MÓDULO DE VALIDACIÓN: TABLAS EUROSTAT
✅ Framework de validación cargado
📊 Tablas EUROSTAT configuradas: 14
🔧 Umbrales: MAX_NULL_PERCENT=5.0%
⏰ Timestamp: 2025-11-20T12:15:10.117027


---

## 1. Identificación de Tablas EUROSTAT

---

In [2]:
# 1. Obtener lista de tablas EUROSTAT en la base de datos
print("=" * 80)
print("IDENTIFICACIÓN DE TABLAS EUROSTAT")
print("=" * 80)

cursor = conn.cursor()
cursor.execute(
    """
    SELECT TABLE_NAME 
    FROM INFORMATION_SCHEMA.TABLES 
    WHERE TABLE_TYPE = 'BASE TABLE' 
      AND TABLE_NAME LIKE 'EUROSTAT_%'
      AND TABLE_NAME NOT LIKE 'VALIDATED_%'
    ORDER BY TABLE_NAME
"""
)

eurostat_tables = [row[0] for row in cursor.fetchall()]

print(f"\n📊 Tablas EUROSTAT encontradas: {len(eurostat_tables)}")
print("\n📋 Lista de tablas:")
for i, table in enumerate(eurostat_tables, 1):
    # Verificar si tiene reglas configuradas
    has_rules = table in EUROSTAT_VALIDATION_RULES
    status_icon = "✅" if has_rules else "⚠️"
    status_text = "CON REGLAS" if has_rules else "SIN REGLAS"
    print(f"   {i:2d}. {status_icon} {table} ({status_text})")

# Estadísticas
tables_with_rules = sum(1 for t in eurostat_tables if t in EUROSTAT_VALIDATION_RULES)
tables_without_rules = len(eurostat_tables) - tables_with_rules

print("\n📈 Resumen:")
print(f"   ✅ Con reglas: {tables_with_rules}")
print(f"   ⚠️  Sin reglas: {tables_without_rules}")

IDENTIFICACIÓN DE TABLAS EUROSTAT

📊 Tablas EUROSTAT encontradas: 14

📋 Lista de tablas:
    1. ✅ EUROSTAT_AROP_Espana (CON REGLAS)
    2. ✅ EUROSTAT_AROP_Ranking (CON REGLAS)
    3. ✅ EUROSTAT_AROP_UE27 (CON REGLAS)
    4. ✅ EUROSTAT_Brecha_Pobreza_Espana (CON REGLAS)
    5. ✅ EUROSTAT_Brecha_Pobreza_Ranking (CON REGLAS)
    6. ✅ EUROSTAT_Brecha_Pobreza_UE27 (CON REGLAS)
    7. ✅ EUROSTAT_Gini_Espana (CON REGLAS)
    8. ✅ EUROSTAT_Gini_Ranking (CON REGLAS)
    9. ✅ EUROSTAT_Gini_UE27 (CON REGLAS)
   10. ✅ EUROSTAT_Impacto_Redistributivo_Espana (CON REGLAS)
   11. ✅ EUROSTAT_Impacto_Redistributivo_UE27 (CON REGLAS)
   12. ✅ EUROSTAT_S80S20_Espana (CON REGLAS)
   13. ✅ EUROSTAT_S80S20_Ranking (CON REGLAS)
   14. ✅ EUROSTAT_S80S20_UE27 (CON REGLAS)

📈 Resumen:
   ✅ Con reglas: 14
   ⚠️  Sin reglas: 0


---

## 2. Función de Validación

---

In [3]:
def validate_eurostat_table(table_name: str, conn, save_report: bool = True) -> dict:
    """
    Valida una tabla EUROSTAT usando el framework y reglas configuradas.
    """
    print("\n" + "=" * 80)
    print(f"VALIDANDO: {table_name}")
    print("=" * 80)
    report = ValidationReport(table_name)
    try:
        df = pd.read_sql(f"SELECT * FROM {table_name}", conn)
        report.records_original = len(df)
    except Exception as e:
        report.add_error(f"Error al cargar tabla: {e}")
        if save_report:
            report.save_json()
            report.save_csv()
        return {
            "table": table_name,
            "status": "ERROR",
            "error": str(e),
            "records_original": 0,
            "records_excluded": 0,
            "errors": 1,
            "warnings": 0,
        }
    rules = get_rules(table_name)
    if not rules:
        report.add_warning("No hay reglas de validación configuradas para esta tabla")
        if save_report:
            report.save_json()
            report.save_csv()
        return {
            "table": table_name,
            "status": "NO_RULES",
            "records_original": len(df),
            "records_excluded": 0,
            "errors": 0,
            "warnings": 1,
        }
    if "expected_columns" in rules or "expected_types" in rules:
        check_schema(
            df,
            expected_columns=rules.get("expected_columns", []),
            expected_types=rules.get("expected_types", {}),
            report=report,
        )
    if "primary_key" in rules:
        pk_columns = rules["primary_key"]
        missing_pk_cols = [col for col in pk_columns if col not in df.columns]
        if missing_pk_cols:
            report.add_warning(f"Columnas de PK no encontradas: {missing_pk_cols}")
        else:
            check_uniqueness(df, primary_key=pk_columns, report=report)
    if "critical_columns" in rules:
        critical_cols = [col for col in rules["critical_columns"] if col in df.columns]
        if critical_cols:
            check_nulls(
                df,
                critical_columns=critical_cols,
                max_null_percent=MAX_NULL_PERCENT,
                report=report,
            )
    # 3b. Validar nulos condicionales
    if "conditional_nulls" in rules:
        check_conditional_nulls(df, rules.get("conditional_nulls", {}), report=report)
    if "range_checks" in rules:
        for column, (min_val, max_val) in rules["range_checks"].items():
            if column in df.columns:
                check_range(
                    df, column=column, min_val=min_val, max_val=max_val, report=report
                )
    if "expected_years" in rules:
        year_column = "Año" if "Año" in df.columns else None
        if year_column:
            check_year_continuity(
                df,
                year_column=year_column,
                expected_years=rules["expected_years"],
                report=report,
            )
    records_excluded = 0
    if "exclude_categories" in rules:
        for column, categories in rules["exclude_categories"].items():
            if column in df.columns:
                for category in categories:
                    count = len(df[df[column] == category])
                    if count > 0:
                        records_excluded += count
                        report.add_warning(
                            f"Encontrados {count} registros de categoría '{category}' ",
                            f"en columna '{column}' (se recomienda excluir en análisis)",
                        )
    report.records_excluded = records_excluded
    if save_report:
        report.save_json()
        report.save_csv()
    return {
        "table": table_name,
        "status": "PASSED" if not report.has_errors() else "FAILED",
        "records_original": len(df),
        "records_excluded": records_excluded,
        "errors": len(report.errors),
        "warnings": len(report.warnings),
    }


print("✅ Función validate_eurostat_table() definida")

✅ Función validate_eurostat_table() definida


---

## 3. Validación Masiva de Tablas EUROSTAT

---

In [4]:
# Validar todas las tablas EUROSTAT
print("=" * 80)
print("VALIDACIÓN MASIVA: TODAS LAS TABLAS EUROSTAT")
print("=" * 80)
print(f"⏰ Inicio: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"📋 Tablas a validar: {len(eurostat_tables)}\n")

# Almacenar resultados
results_eurostat = []

for i, table in enumerate(eurostat_tables, 1):
    print(f"\n[{i}/{len(eurostat_tables)}] Procesando: {table}")
    result = validate_eurostat_table(table, conn, save_report=True)
    results_eurostat.append(result)

print(f"\n⏰ Fin: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

VALIDACIÓN MASIVA: TODAS LAS TABLAS EUROSTAT
⏰ Inicio: 2025-11-20 12:15:10
📋 Tablas a validar: 14


[1/14] Procesando: EUROSTAT_AROP_Espana

VALIDANDO: EUROSTAT_AROP_Espana


[REPORT] Report saved: C:\Users\mario\Desktop\Projects\desigualdad_social_etl\notebooks\00_etl\..\..\data\validated\logs\EUROSTAT_AROP_Espana_20251120_121510.json
[REPORT] Report saved: C:\Users\mario\Desktop\Projects\desigualdad_social_etl\notebooks\00_etl\..\..\data\validated\logs\EUROSTAT_AROP_Espana_20251120_121510.csv

[2/14] Procesando: EUROSTAT_AROP_Ranking

VALIDANDO: EUROSTAT_AROP_Ranking
[REPORT] Report saved: C:\Users\mario\Desktop\Projects\desigualdad_social_etl\notebooks\00_etl\..\..\data\validated\logs\EUROSTAT_AROP_Ranking_20251120_121510.json
[REPORT] Report saved: C:\Users\mario\Desktop\Projects\desigualdad_social_etl\notebooks\00_etl\..\..\data\validated\logs\EUROSTAT_AROP_Ranking_20251120_121510.csv

[3/14] Procesando: EUROSTAT_AROP_UE27

VALIDANDO: EUROSTAT_AROP_UE27
[REPORT] Report saved: C:\Users\mario\Desktop\Projects\desigualdad_social_etl\notebooks\00_etl\..\..\data\validated\logs\EUROSTAT_AROP_UE27_20251120_121510.json
[REPORT] Report saved: C:\Users\mario\Des

C:\Users\mario\AppData\Local\Temp\ipykernel_13408\2794360249.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f'SELECT * FROM {table_name}', conn)
C:\Users\mario\AppData\Local\Temp\ipykernel_13408\2794360249.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f'SELECT * FROM {table_name}', conn)
C:\Users\mario\AppData\Local\Temp\ipykernel_13408\2794360249.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f'SELECT * FROM {table_name}', conn)
C:\Users\mario\AppData\Lo

---

## 4. Resumen Ejecutivo

---

In [5]:
# Resumen consolidado
print("=" * 80)
print("RESUMEN EJECUTIVO: VALIDACIÓN TABLAS EUROSTAT")
print("=" * 80)

df_results = pd.DataFrame(results_eurostat)

print(f"\n📊 TABLAS PROCESADAS: {len(results_eurostat)}")
print("\n" + "=" * 80)
print("DETALLE POR TABLA")
print("=" * 80)
print(df_results.to_string(index=False))

# Estadísticas globales
total_errors = df_results["errors"].sum()
total_warnings = df_results["warnings"].sum()
total_original = df_results["records_original"].sum()
total_excluded = df_results["records_excluded"].sum()

# Contar estados
passed = len(df_results[df_results["status"] == "PASSED"])
failed = len(df_results[df_results["status"] == "FAILED"])
no_rules = len(df_results[df_results["status"] == "NO_RULES"])
errors = len(df_results[df_results["status"] == "ERROR"])

print("\n" + "=" * 80)
print("ESTADÍSTICAS GLOBALES")
print("=" * 80)
print(f"📊 Registros totales: {total_original:,}")
print(
    f"✂️  Registros con advertencia de exclusión: {total_excluded:,} ({total_excluded/total_original*100:.2f}% if total_original > 0 else 0)"
)
print(f"❌ Total errores: {total_errors}")
print(f"⚠️  Total advertencias: {total_warnings}")

print("\n📈 Estado de validación:")
print(f"   ✅ PASSED: {passed}")
print(f"   ❌ FAILED: {failed}")
print(f"   ⚠️  NO_RULES: {no_rules}")
print(f"   🚫 ERROR: {errors}")

if total_errors == 0 and failed == 0:
    print("\n🎉 ¡VALIDACIÓN COMPLETADA SIN ERRORES CRÍTICOS!")
else:
    print(f"\n⚠️  Se encontraron {total_errors} errores en {failed} tablas.")

print("\n" + "=" * 80)
print("REPORTES GENERADOS")
print("=" * 80)
print("📁 Ubicación: ../../data/validated/logs/")
print("📄 Formato: JSON y CSV por cada tabla")
print(
    f"✅ Total reportes: {len(results_eurostat) * 2} archivos ({len(results_eurostat)} JSON + {len(results_eurostat)} CSV)"
)

print("\n" + "=" * 80)
print("🎯 PRÓXIMOS PASOS")
print("=" * 80)
print("1. Revisar reportes JSON/CSV en data/validated/logs/")
print("2. Corregir errores críticos si los hay (tablas FAILED)")
print("3. Configurar reglas para tablas sin validación (NO_RULES)")
print(
    "4. Si completaste 02a_validacion_INE.ipynb, ejecutar 02c_validacion_integracion.ipynb"
)
print("=" * 80)

RESUMEN EJECUTIVO: VALIDACIÓN TABLAS EUROSTAT

📊 TABLAS PROCESADAS: 14

DETALLE POR TABLA
                                 table status  records_original  records_excluded  errors  warnings
                  EUROSTAT_AROP_Espana PASSED                10                 0       0         0
                 EUROSTAT_AROP_Ranking PASSED               397                 0       0         0
                    EUROSTAT_AROP_UE27 PASSED                10                 0       0         1
        EUROSTAT_Brecha_Pobreza_Espana PASSED                10                 0       0         1
       EUROSTAT_Brecha_Pobreza_Ranking PASSED               378                 0       0         1
          EUROSTAT_Brecha_Pobreza_UE27 PASSED                10                 0       0         1
                  EUROSTAT_Gini_Espana PASSED                10                 0       0         1
                 EUROSTAT_Gini_Ranking PASSED               365                 0       0         0
          

In [6]:
# Cerrar conexión
conn.close()
print("✅ Conexión a SQL Server cerrada")

✅ Conexión a SQL Server cerrada
